# Extended Iterable Unpacking

本PEP对可迭代对象的拆包方式进行了扩展。

本PEP于Python3.0版本接受

## 简要概述

本PEP对可迭代对象的拆包语法（iterable unpacking syntax）进行了扩展：允许使用一个额外的变量接收多余拆包数据。如下示例：
```Python
>>> a, *b, c = range(5)
>>> a
0
>>> c
4
>>> b
[1, 2, 3]
```

上述例子中，共有三个变量接收拆包结果，a变量接收第一个元素，c变量接收最后一个元素，\*b则接收剩余的元素并记入一个列表

## 出发点

对于一个序列类型，可能需要按照顺序获取前几个元素以及剩余元素。在本PEP提出之前可以依照如下方式获取：
```Python
first_element, rest_elements = seq[0], seq[1:]
```
在本PEP提出后可以使用如下语法
```Python
first_element, *rest_elements = seq
```

毫无疑问，使用新的拆包语法更简介。这一方案的简洁性在更复杂的切片操作中更明显，例如希望使用三个变量分别存放首尾元素以及剩余元素，在本PEP的语法支持下仅需要写为：
```Python
first_element, *rest_elements, last_element = seq
```

## 注意事项

* 对于一个赋值语句，最多只能有一个\*描述的变量执行上述功能

* 若迭代器中含有的元素个数少于需要的数量（除\*描述的变量外其他变量的数量），会直接报错（ValueError）

* 仅有一个变量的赋值语句使用上述语法会直接报错
```Python
>>> *a = range(5)
SyntaxError
>>> *a, = range(5)
>>> a
>>> [0, 1, 2, 3, 4]
```

* 支持对元组进行操作，但是会返回列表类型

* 对于任何可迭代的对象，上述语法均会返回list类型

## 实现细节

新增UNPACK_EX字节码指令，该指令要求获取位于\*描述的变量前的变量数量以及位于其后的变量数量。对于没有使用\*的赋值语句，默认使用UNPACK_ITERABLE字节码指令执行拆包

具体实现
1. 首先为位于\*描述的变量前的变量按顺序赋值
2. 将剩余元素记入一个列表
3. 为位于\*描述的变量后的变量赋值
4. 将剩余元素赋予\*描述的变量

## 其他细节

本PEP的[邮件列表](https://mail.python.org/pipermail/python-3000/2007-May/007198.html)展示了关于本PEP的争论要点。这些争论的结论在最终成文的PEP 3132中有所体现。

Neville Grech Neville Grech提到了两个非常关键的问题：

1. 对于类似于\*a = range(5)这样的用法应当如何响应？
2. 对于赋值语句\*a, b = tuple(range(5))，a的类型应当是什么？

本PEP对上述问题的解答：

1. 毫无疑问会报错，上述的拆包语法不支持这种写法
2. a的类型为列表，即在赋值语句中\*描述的变量的类型不受原可迭代对象的影响。

对于第二个问题，看起来和原类型保持一致是有益的。但是考虑到项目中使用的可迭代对象可能千奇百怪，若和原类型保持一致可能导致一些奇怪的bug

### 关于\*可使用的位置

Brett Cannon则提议仅限赋值语句左侧最后一个变量使用\*描述，一方面本语法的处理难度更低，另一方面Brett认为那些既需要提取前几个元素又需要提取后几个元素的场合很少。

从最终成文的PEP 3132来看，Brett的意见没有被接受。PEP 3132允许使用\*描述的变量出现在赋值语句左侧的任意位置。

有意思的是，Guido van Rossum也认为允许\*描述的变量出现在赋值语句左侧的任何位置会导致混乱（That sounds messy），而仅允许最后一个变量使用更易于管理。

此外，Guido van Rossum也建议在赋值语句中至少有一个逗号的情况下使用\*才正确。

这一建议使得赋值语句： \*a = range(5)会报错，而赋值语句：\*a, = range(5)则是可行的。

### 关于返回值

本PEP最大的争议就是\*描述的变量应该是什么类型？总的来说，有三类观点：

1. 无论如何都返回list类型
2. 赋值语句右侧的变量是什么类型就返回什么类型
3. 区别看待，list返回list，tuple返回tuple，其他的返回一个迭代器

观点1是最为保险的观点，也是本PEP最后采用的观点，虽然会造成一些不便（例如对于赋值语句a, \*b="Hello"，在本PEP下，b=['e', 'l', 'l', '0']，但是返回b="ello"毫无疑问更方便）

观点2描述的方案当然是最理想的，但是也是最容易出错的处理方法。因为很难保证兼容所有的类型，尤其是对于自定义的类型。

观点3的方案则是区别看待内置类型和其他自定义类型。这样处理增加了复杂度，但是也是有可取的之处，例如对于内置类型可以使用优化后的处理流程。